In [1]:
import re
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
def select_gpu(N):
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[N], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)    

In [3]:
select_gpu(0)
epochs = 1024
batch_size = 1024
model_name = 'bert-chinese'
embedding_path = '/home/Danny/pretrain_model/{}'.format(model_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 1 Logical GPUs


In [4]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [5]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')

In [6]:
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')

In [7]:
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [8]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [9]:
df = pd.DataFrame({'x': x, 'y': y})
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
76157,"[醫, 師, ：, 他, 有, 在, 騎, 腳, 踏, 車]","[O, O, O, O, O, O, O, O, O, O]"
76158,"[民, 眾, ：, 恩]","[O, O, O, O]"
76159,"[醫, 師, ：, 騎, 腳, 踏, 車, 可, 以]","[O, O, O, O, O, O, O, O, O]"
76160,"[小, 美]","[B-name, I-name]"


In [10]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

/home/Danny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/Danny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
75713,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
75714,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
75716,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
75717,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [11]:
x = df['x'].to_list()
y = df['y'].to_list()

In [12]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

20172 20172
5044 5044
6305 6305


In [ ]:
%%time
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BertEmbedding
kashgari.config.use_cudnn_cell = True
bert_embedding = BertEmbedding(embedding_path, 
                           sequence_length='auto',
                           trainable=True,
                           task='kashgari.LABELING', 
                          )
model = BiLSTM_CRF_Model(bert_embedding)
history = model.fit(train_x,
                    train_y,
                    valid_x,
                    valid_y,
                    epochs=epochs,
                    batch_size=batch_size,
                   )

2020-12-15 15:26:38,833 [DEBUG] kashgari - ------------------------------------------------
2020-12-15 15:26:38,834 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-15 15:26:38,834 [DEBUG] kashgari - config_path       : /home/Danny/pretrain_model/bert-chinese/bert_config.json
2020-12-15 15:26:38,835 [DEBUG] kashgari - vocab_path      : /home/Danny/pretrain_model/bert-chinese/vocab.txt
2020-12-15 15:26:38,835 [DEBUG] kashgari - checkpoint_path : /home/Danny/pretrain_model/bert-chinese/bert_model.ckpt
2020-12-15 15:26:38,836 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[

Epoch 1/1024
19/19 [==============================] - 18s 933ms/step - loss: 4.4436 - accuracy: 0.8508 - val_loss: 29.2580 - val_accuracy: 0.9693
Epoch 2/1024
19/19 [==============================] - 13s 705ms/step - loss: 1.2681 - accuracy: 0.9664 - val_loss: 29.1579 - val_accuracy: 0.9707
Epoch 3/1024
19/19 [==============================] - 13s 700ms/step - loss: 0.8948 - accuracy: 0.9694 - val_loss: 29.1220 - val_accuracy: 0.9723
Epoch 4/1024
19/19 [==============================] - 13s 700ms/step - loss: 0.7105 - accuracy: 0.9736 - val_loss: 29.1860 - val_accuracy: 0.9753
Epoch 5/1024
19/19 [==============================] - 13s 703ms/step - loss: 0.6068 - accuracy: 0.9758 - val_loss: 28.9306 - val_accuracy: 0.9761
Epoch 6/1024
19/19 [==============================] - 13s 701ms/step - loss: 0.5324 - accuracy: 0.9774 - val_loss: 28.9663 - val_accuracy: 0.9784
Epoch 7/1024
19/19 [==============================] - 13s 704ms/step - loss: 0.4806 - accuracy: 0.9782 - val_loss: 28.8168 -

Epoch 57/1024
19/19 [==============================] - 13s 710ms/step - loss: 0.0606 - accuracy: 0.9966 - val_loss: 27.8947 - val_accuracy: 0.9820
Epoch 58/1024
19/19 [==============================] - 14s 711ms/step - loss: 0.0613 - accuracy: 0.9963 - val_loss: 27.8141 - val_accuracy: 0.9844
Epoch 59/1024
19/19 [==============================] - 14s 711ms/step - loss: 0.0602 - accuracy: 0.9965 - val_loss: 27.9176 - val_accuracy: 0.9828
Epoch 60/1024
19/19 [==============================] - 14s 711ms/step - loss: 0.0576 - accuracy: 0.9967 - val_loss: 27.8348 - val_accuracy: 0.9841
Epoch 61/1024
19/19 [==============================] - 14s 722ms/step - loss: 0.0551 - accuracy: 0.9969 - val_loss: 27.9861 - val_accuracy: 0.9839
Epoch 62/1024
19/19 [==============================] - 14s 718ms/step - loss: 0.0533 - accuracy: 0.9968 - val_loss: 27.9189 - val_accuracy: 0.9827
Epoch 63/1024
19/19 [==============================] - 13s 709ms/step - loss: 0.0499 - accuracy: 0.9972 - val_loss: 27

19/19 [==============================] - 13s 705ms/step - loss: 0.0219 - accuracy: 0.9986 - val_loss: 27.2985 - val_accuracy: 0.9831
Epoch 113/1024
19/19 [==============================] - 13s 705ms/step - loss: 0.0223 - accuracy: 0.9985 - val_loss: 27.2515 - val_accuracy: 0.9838
Epoch 114/1024
19/19 [==============================] - 13s 705ms/step - loss: 0.0247 - accuracy: 0.9984 - val_loss: 27.4080 - val_accuracy: 0.9832
Epoch 115/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0213 - accuracy: 0.9986 - val_loss: 27.4030 - val_accuracy: 0.9830
Epoch 116/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0224 - accuracy: 0.9986 - val_loss: 27.2270 - val_accuracy: 0.9831
Epoch 117/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0223 - accuracy: 0.9985 - val_loss: 27.3155 - val_accuracy: 0.9835
Epoch 118/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0227 - accuracy: 0.9984 - val_loss: 27.2384 - 

19/19 [==============================] - 13s 708ms/step - loss: 0.0156 - accuracy: 0.9989 - val_loss: 26.8411 - val_accuracy: 0.9814
Epoch 168/1024
19/19 [==============================] - 13s 708ms/step - loss: 0.0159 - accuracy: 0.9988 - val_loss: 26.7619 - val_accuracy: 0.9809
Epoch 169/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0165 - accuracy: 0.9989 - val_loss: 26.7501 - val_accuracy: 0.9791
Epoch 170/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0146 - accuracy: 0.9989 - val_loss: 26.7182 - val_accuracy: 0.9797
Epoch 171/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0162 - accuracy: 0.9987 - val_loss: 26.8211 - val_accuracy: 0.9807
Epoch 172/1024
19/19 [==============================] - 14s 725ms/step - loss: 0.0150 - accuracy: 0.9989 - val_loss: 26.7671 - val_accuracy: 0.9813
Epoch 173/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0146 - accuracy: 0.9989 - val_loss: 26.8379 - 

19/19 [==============================] - 14s 716ms/step - loss: 0.0133 - accuracy: 0.9988 - val_loss: 26.4453 - val_accuracy: 0.9799
Epoch 223/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0126 - accuracy: 0.9989 - val_loss: 26.2347 - val_accuracy: 0.9788
Epoch 224/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0133 - accuracy: 0.9988 - val_loss: 26.4322 - val_accuracy: 0.9795
Epoch 225/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0121 - accuracy: 0.9990 - val_loss: 26.3981 - val_accuracy: 0.9795
Epoch 226/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0107 - accuracy: 0.9990 - val_loss: 26.2946 - val_accuracy: 0.9820
Epoch 227/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0123 - accuracy: 0.9989 - val_loss: 26.2546 - val_accuracy: 0.9805
Epoch 228/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0122 - accuracy: 0.9989 - val_loss: 26.3326 - 

19/19 [==============================] - 13s 706ms/step - loss: 0.0101 - accuracy: 0.9990 - val_loss: 25.8693 - val_accuracy: 0.9726
Epoch 278/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0111 - accuracy: 0.9989 - val_loss: 25.8517 - val_accuracy: 0.9730
Epoch 279/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0100 - accuracy: 0.9990 - val_loss: 25.8408 - val_accuracy: 0.9754
Epoch 280/1024
19/19 [==============================] - 13s 708ms/step - loss: 0.0094 - accuracy: 0.9991 - val_loss: 25.8854 - val_accuracy: 0.9756
Epoch 281/1024
19/19 [==============================] - 14s 717ms/step - loss: 0.0099 - accuracy: 0.9990 - val_loss: 25.8579 - val_accuracy: 0.9756
Epoch 282/1024
19/19 [==============================] - 14s 717ms/step - loss: 0.0099 - accuracy: 0.9990 - val_loss: 25.8562 - val_accuracy: 0.9773
Epoch 283/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0102 - accuracy: 0.9990 - val_loss: 25.7578 - 

19/19 [==============================] - 13s 709ms/step - loss: 0.0116 - accuracy: 0.9987 - val_loss: 25.2843 - val_accuracy: 0.9646
Epoch 333/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0115 - accuracy: 0.9989 - val_loss: 25.1245 - val_accuracy: 0.9691
Epoch 334/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0108 - accuracy: 0.9988 - val_loss: 25.0888 - val_accuracy: 0.9721
Epoch 335/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0102 - accuracy: 0.9989 - val_loss: 24.9532 - val_accuracy: 0.9743
Epoch 336/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0106 - accuracy: 0.9988 - val_loss: 25.0338 - val_accuracy: 0.9724
Epoch 337/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0105 - accuracy: 0.9989 - val_loss: 25.0240 - val_accuracy: 0.9715
Epoch 338/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0117 - accuracy: 0.9988 - val_loss: 24.9555 - 

19/19 [==============================] - 13s 708ms/step - loss: 0.0071 - accuracy: 0.9990 - val_loss: 24.5070 - val_accuracy: 0.9390
Epoch 388/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0065 - accuracy: 0.9991 - val_loss: 24.4395 - val_accuracy: 0.9061
Epoch 389/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0076 - accuracy: 0.9989 - val_loss: 24.3891 - val_accuracy: 0.9027
Epoch 390/1024
19/19 [==============================] - 13s 706ms/step - loss: 0.0078 - accuracy: 0.9989 - val_loss: 24.4187 - val_accuracy: 0.7837
Epoch 391/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0074 - accuracy: 0.9990 - val_loss: 24.4537 - val_accuracy: 0.7832
Epoch 392/1024
19/19 [==============================] - 13s 707ms/step - loss: 0.0075 - accuracy: 0.9991 - val_loss: 24.4704 - val_accuracy: 0.7860
Epoch 393/1024
19/19 [==============================] - 13s 705ms/step - loss: 0.0084 - accuracy: 0.9989 - val_loss: 24.4406 - 

In [ ]:
model.evaluate(test_x, test_y)
model_path = 'model/{}_epoch_{}_batch_{}'.format(model_name, epochs, batch_size)
model.save(model_path)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy', 'loss', 'val_loss'])
plt.show()

In [ ]:
# 需要儲存article_id, 轉成dataframe
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [ ]:
def predict_NER(text):
    x_list = list()    
    text_list = re.split('\uff0c|\u3002|\uff1f', text)
    for article in text_list:
        x_list.append([i for i in article])
    y_list_list = model.predict(x_list)
    y_list = list()
    for sentence in y_list_list:
        for word in sentence:
            y_list.append(word)
        y_list.append('O') # append(，。？)
    y_list = y_list[:-1]
    return y_list

In [ ]:
def output_NER(article_id, text, y_list):
    output_str = str()
    flag = False
    for i, j in enumerate(y_list):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            entity = '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
            output_str += entity
    return output_str

In [ ]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
output_str = "article_id\tstart_position\tend_position\tentity_text\tentity_type\n"

for article_id, text in zip(df['article_id'], df['text']):
    x_list = [word for word in text]
    y_list = predict_NER(text)
    output_str += output_NER(article_id, text, y_list)

In [ ]:
output_path = 'output/{}_epoch_{}_batch_{}.tsv'.format(model_name, epochs, batch_size)
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output_str)

In [ ]:
df = pd.read_csv(output_path, sep='\t')
df